# Lana Culp - 11.20.2024 - Final Project: Data in Hand
## Overview
I will be dissecting Overwatch 2 data based off of the hero statistics as of the year 2023. I will be trying to decipher which hero is the most worth learning to play based off of the win rate percentage. 

### References
I utilized statistics derrived from Kaggle (https://www.kaggle.com/datasets/mykhailokachan/overwatch-2-statistics?resource=download) which pulled data from the Heroes section on Overbuff (https://www.overbuff.com/heroes) which is an Overwatch data website that holds player statistics which allows players to look through the data and hopefully use that information to improve their own gameplay. The data was also listed on GitHub (https://github.com/mias1wot/Get-statistics-from-Overbuff). 

In [70]:
import urllib
import requests
import pandas as pd 
import json
import base64
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from scipy import stats
import statsmodels.formula.api as smf
import numpy as np

### Overwatch 2 Hero Statistics 
Currently the hero statistics are organized by hero and skill tier. Each hero is listed with the statistics for each skill tier the hero was played in. The current skill tiers available listed from "easiest" to "hardest" are: bronze, silver, gold, platinum, diamond, master, and grandmaster. There is then an overall tier labeled "all" that describes the hero statistics across all of the skill tiers. With this data being collected from a third party source, it may not be exact information, but it is mostly accurate, and this is my interpretation of the data provided. 

In [76]:
ow_stats = pd.read_csv('ow2_quickplay_heroes_stats__2023-05-06.csv')
#ow_stats.head()

In [64]:
#ow_stats.keys()

### Tidy Data
I chose to drop a majority of columns due to them being extremely hero specific. I wanted to showcase the data that pertained to mostly all of the heros. 

In [69]:
#ow_stats_tidy.keys()

In [72]:
ow_stats_tidy = ow_stats.drop(columns = ['Jagged Blade Accuracy, %', 'Carnage Kills / 10min', 'Wound Uptime, %',
       'Rampage Kills / 10min', 'Focusing Beam Accuracy, %',
       'Focusing Beam Kills / 10min', 'Sticky Bomb Accuracy, %',
       'Sticky Bomb Kills / 10min', 'Duplicate Kills / 10min', 'Players Teleported / 10min', 'Sentry Turret Kills / 10min',
       'Hammer Kills / 10min', 'Turret Kills / 10min',
       'Molten Core Kills / 10min', 'Pummel Accuracy, %',
       'Pummel Kills / 10min', 'Ravenous Vortex Kills / 10min',
       'Annihilation Kills / 10min', 'EMP Kills / 10min', 'Grappling Claw Kills / 10min',
       'Piledriver Kills / 10min', 'Charged Shot Accuracy, %',
       'Charged Crit Accuracy, %', 'Charged Shot Kills / 10min',
       'Disruptor Shot Kills / 10min', 'Overclock Kills / 10min',
       'Average Damage Multiplier', 'Blizzard Kills / 10min', 'Jump Pack Kills / 10min',
       'Primal Rage Kills / 10min', 'Recon Kills / 10min',
       'Assault Kills / 10min', 'Tactical Grenade Kills / 10min',
       'Artillery Kills / 10min', 'Enemies Hacked / 10min',
       'Low Health Teleports / 10min', 'Overhealth / 10min', 'Gravitic Flux Kills / 10min',
       'Rocket Punch Kills / 10min', 'Seismic Slam Kills / 10min',
       'Meteor Strike Kills / 10min', 'Inspire Uptime',
       'Icicle Accuracy, % / 10min', 'Icicle Crit Accuracy, % / 10min',
       'Enemies Frozen / 10min', 'Healing Accuracy', 'Deaths Prevented / 10min',
       'Amp Matrix Assists / 10min', 'Energy Javelin Kills / 10min',
       'Javelin Spin Kills / 10min', 'Terra Surge Kills / 10min', 'Airtime, %',
       'Barrage Kills / 10min', 'Accretion Kills / 10min', 'Pulse Bomb Kills / 10min', 'Chain Hook Kills / 10min',
       'Chain Hook Accuracy, %', 'Whole Hog Kills / 10min',
       'Primary Accuracy, %', 'High Energy Kills / 10min', 'Average Energy, %',
       'Graviton Surge Kills / 10min', 'Death Blossom Kills / 10min', 'Direct Hits',
       'Concussion Mine Kills / 10min', 'Enemies Trapped / 10min',
       'Coach Gun Kills / 10min', 'Dynamite Kills / 10min',
       'Bob Kills / 10min', 'Self Healing / 10min',
       'Low Health Recalls / 10min', 'Pulse Bombs Stuck / 10min', 'Tactical Visor Kills / 10min',
       'Charged Volley Accuracy, %', 'Charged Volley Kills / 10min',
       'Transcendence Healing / 10min', 'Knockbacks / 10min',
       'Sound Barriers / 10min', 'Micro Missile Kills, % / 10min',
       'Call Mech Kills / 10min', 'Self-Destruct Kills / 10min', 'Fire Strike Kills / 10min', 'Earthshatter Kills / 10min',
       'Long Range Kills / 10min', 'Storm Arrow Kills / 10min',
       'Recon Assists / 10min', 'Dragonstrike Kills / 10min',
       'Venom Mine Kills / 10min', 'Helix Rocket Accuracy, %',
       'Helix Rocket Kills / 10min', 'Effects Cleansed / 10min',
       'Kitsune Rush Assists / 10min', 'Secondary Accuracy, %',
       'Biotic Orb Kills / 10min', 'Ally Coalescence Efficiency, %',
       'Enemy Coalescence Efficiency, %', 'Coalescence Kills / 10min',
       'Objective Contest Time / 10min', 'Charge Kills / 10min', 'Crit Accuracy, %', 'Deadeye Kills / 10min',
       'Magnetic Grenade Attaches', 'Magnetic Grenade Kills / 10min',
       'Swift Strike Resets / 10min', 'Damage Reflected / 10min',
       'Ultimates Reflected / 10min', 'Dragonblade Kills / 10min',
       'Kunai Kills / 10min', 'Biotic Grenade Kills / 10min',
       'Enemies Slept / 10min', 'Healing Amplified / 10min',
       'Nano Boost Assists / 10min', 'Blaster Kills / 10min',
       'Damage Amplified / 10min', 'Players Resurrected / 10min'])
ow_stats_tidy.head()

,Hero,Skill Tier,KDA Ratio,"Pick Rate, %","Win Rate, %",Eliminations / 10min,Objective Kills / 10min,Objective Time / 10min,Damage / 10min,Healing / 10min,Deaths / 10min,"Scoped Accuracy, %",Final Blows / 10min,Solo Kills / 10min,"Weapon Accuracy, %",Role
0,Ana,All,4.91,7.75,48.73,9.80,3.93,53,2641,7278.0,6.28,55.0,NaN,NaN,NaN,Support
1,Ana,Bronze,4.29,6.05,47.17,9.12,3.75,53,2483,6819.0,6.51,53.0,NaN,NaN,NaN,Support
2,Ana,Silver,4.33,5.77,47.27,9.19,3.80,53,2505,6829.0,6.52,53.0,NaN,NaN,NaN,Support
3,Ana,Gold,4.45,6.01,47.54,9.37,3.86,54,2542,6958.0,6.47,54.0,NaN,NaN,NaN,Support
4,Ana,Platinum,4.70,7.06,47.94,9.60,3.91,53,2575,7168.0,6.36,55.0,NaN,NaN,NaN,Support


### Sorting Values
I decided to sort the data with the highest win percentage at the top just to see which hero is deemed the most likely to win when played in games, and from the data, it seems that the support hero Brigette has the highest win percentage when played at the Grandmaster skill tier level. 

In [73]:
ow_stats_tidy.sort_values('Win Rate, %', ascending = False).head(10)

,Hero,Skill Tier,KDA Ratio,"Pick Rate, %","Win Rate, %",Eliminations / 10min,Objective Kills / 10min,Objective Time / 10min,Damage / 10min,Healing / 10min,Deaths / 10min,"Scoped Accuracy, %",Final Blows / 10min,Solo Kills / 10min,"Weapon Accuracy, %",Role
199,Brigitte,Grandmaster,4.72,2.10,56.36,13.97,5.36,80,3831,7336.0,7.50,NaN,NaN,NaN,NaN,Support
198,Brigitte,Master,4.67,2.02,56.32,13.98,5.39,80,3829,7326.0,7.55,NaN,NaN,NaN,NaN,Support
197,Brigitte,Diamond,4.69,2.00,56.22,14.03,5.43,81,3841,7320.0,7.59,NaN,NaN,NaN,NaN,Support
250,Symmetra,Silver,3.66,1.57,56.18,18.92,8.33,56,7046,NaN,7.68,NaN,8.62,1.77,NaN,Damage
249,Symmetra,Bronze,3.59,1.60,56.13,18.72,8.25,56,6943,NaN,7.66,NaN,8.51,1.77,NaN,Damage
192,Brigitte,All,4.80,1.83,56.10,14.04,5.40,81,3873,7349.0,7.62,NaN,NaN,NaN,NaN,Support
196,Brigitte,Platinum,4.63,1.79,56.08,13.95,5.45,82,3817,7233.0,7.69,NaN,NaN,NaN,NaN,Support
195,Brigitte,Gold,4.46,1.64,56.01,13.69,5.42,83,3737,7076.0,7.82,NaN,NaN,NaN,NaN,Support
255,Symmetra,Grandmaster,3.63,1.76,55.99,19.08,8.05,53,7212,NaN,7.65,NaN,8.83,1.84,NaN,Damage
194,Brigitte,Silver,4.35,1.66,55.94,13.42,5.36,84,3653,6974.0,7.87,NaN,NaN,NaN,NaN,Support


### Data in Hand Conclusion
At this point in the final project, I have access to all my data and have tidied the data. I can clearly understand the structure of 